# Hyperparameter Tuning and Model Comparison

**Dataset**: MNIST (handwritten digits)  
**Framework**: Keras (TensorFlow backend)  
**Goal**: Compare 3 model configurations by tuning:
- Learning rate
- Batch size
- Number of hidden layers

We'll use **TensorBoard** for logging.

In [1]:
# !pip install tensorflow tensorboard

In [2]:
# Import libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import datetime
import os

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess
x_train = x_train.reshape(-1, 28*28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0

# One-hot encode labels
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

print(f"Training samples: {x_train.shape[0]}")
print(f"Test samples: {x_test.shape[0]}")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training samples: 60000
Test samples: 10000


## Define Model Builder Function

In [4]:
def build_model(num_layers=1, units=128, learning_rate=0.001):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(784,)))

    for _ in range(num_layers):
        model.add(keras.layers.Dense(units, activation='relu'))

    model.add(keras.layers.Dense(10, activation='softmax'))

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

## Experiment Configurations

In [6]:
configs = [
    {"name": "base",        "lr": 0.001, "batch_size": 32,  "layers": 1},
    {"name": "high_lr",     "lr": 0.01,  "batch_size": 32,  "layers": 1},
    {"name": "deep",        "lr": 0.001, "batch_size": 64,  "layers": 3},
]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
os.makedirs(log_dir, exist_ok=True)

## Train Models with TensorBoard Logging

In [ ]:
for cfg in configs:
    print(f"\nTraining: {cfg['name']}")

    model = build_model(
        num_layers=cfg["layers"],
        learning_rate=cfg["lr"]
    )

    tensorboard_callback = keras.callbacks.TensorBoard(
        log_dir=f"{log_dir}/{cfg['name']}",
        histogram_freq=1
    )

    model.fit(
        x_train, y_train,
        batch_size=cfg["batch_size"],
        epochs=5,
        validation_data=(x_test, y_test),
        callbacks=[tensorboard_callback],
        verbose=1
    )


Training: base
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.8778 - loss: 0.4299 - val_accuracy: 0.9579 - val_loss: 0.1389
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9637 - loss: 0.1242 - val_accuracy: 0.9712 - val_loss: 0.0984
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9770 - loss: 0.0764 - val_accuracy: 0.9703 - val_loss: 0.0952
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9815 - loss: 0.0601 - val_accuracy: 0.9737 - val_loss: 0.0861
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9856 - loss: 0.0479 - val_accuracy: 0.9760 - val_loss: 0.0793

Training: high_lr
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8931 - loss: 0.3456 - val_accuracy: 0.9489 - val_loss: 0.1661
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9546 - loss: 0.1606 - val_accuracy: 0.9550 - val_loss: 0.1758
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - 

## Launch TensorBoard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

Open http://localhost:6006